In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
filename = "X:/georisk/HaRIA_B_Wind/projects/qfes_swha/data/derived/exposure/2020/SEQ_ResidentialExposure_NEXIS_2020_M4.csv"
df = pd.read_csv(filename, sep=",",header=0, index_col=0, skipinitialspace=True)

# Filter out records that do not have an M4 value. 
df = df[df['M4'].notnull()]
df.info()

In [ ]:
def buildingClass(df, classes, thresholds, AS1170='C'):

    for thres, cls in zip(thresholds, classes):
        idx = np.where((df['M4'] >= thres) & (df['WIND_REGION_CLASSIFCATION'] == AS1170))[0]
        df['AS4055_CLASS'][idx] = cls
        
    return df


# Apply a basic AS4055 class, based on the M4 value

thresholds = [0.0, 0.8668, 1.008, 1.1811, 1.4406]
classes = ['N1', 'N2', 'N3', 'N4', 'N5']
df = buildingClass(df, classes, thresholds, 'A')


thresholds = [0.0, 0.8278, 0.973, 1.147, 1.3412]
classes = ['N2', 'N3', 'N4', 'N5', 'N6']
df = buildingClass(df, classes, thresholds, 'B')

# Region C
thresholds = [0.0, 0.8278, 0.973, 1.147]
classes = ['C1', 'C2', 'C3', 'C4']
df = buildingClass(df, classes, thresholds, 'C')

# Region D
thresholds = [0.0, 0.8088, 0.9275]
classes = ['C1', 'C2', 'C3']
df = buildingClass(df, classes, thresholds, 'D')


In [ ]:
np.sort(df.YEAR_BUILT.unique())

In [ ]:
np.sort(df.WALL_TYPE.unique())

In [ ]:
np.sort(df.ROOF_TYPE.unique())

In [ ]:
classes = ['N1', 'N2', 'N3', 'N4', 'N5', 'N6']
curves = ['dw309', 'dw310', 'dw311', 'dw312', 'dw313', 'dw314']
filter = df['YEAR_BUILT'].map(lambda x: x not in ['1982 - 1996', '1997 - present'])
for cls, curve in zip(classes, curves):
    idx = np.where(df['AS4055_CLASS'] == cls)[0]
    df['WIND_VULNERABILITY_FUNCTION_ID'][idx] = curve

In [ ]:
df['WIND_VULNERABILITY_FUNCTION_ID'][df.YEAR_BUILT.isin(['1840 - 1890', '1891 - 1913', '1914 - 1946','1947 - 1961', '1962 - 1981'])] = 'dw310'
df['WIND_VULNERABILITY_FUNCTION_ID'][(df.ROOF_TYPE=='Fibro / asbestos cement sheeting') & 
                                     (df.WALL_TYPE.isin(['Fibro / asbestos cement sheeting', 'Timber'])) &
                                     (df.YEAR_BUILT.isin(['1962 - 1981']))] = 'dw309'

df['WIND_VULNERABILITY_FUNCTION_ID'][(df.ROOF_TYPE=='Metal Sheeting') & 
                                     (df.WALL_TYPE.isin([['Fibro / asbestos cement sheeting', 'Timber']])) &
                                     (df.YEAR_BUILT.isin(['1947 - 1961', '1962 - 1981']))] = 'dw309'
df['WIND_VULNERABILITY_FUNCTION_ID'][(df.ROOF_TYPE=='Metal Sheeting') & 
                                     (df.YEAR_BUILT.isin(['1840 - 1890', '1891 - 1913', '1914 - 1946'])) & 
                                     (df.WALL_TYPE.isin(['Timber', 'Fibro / asbestos cement sheeting']))] = 'dw315'
df['WIND_VULNERABILITY_FUNCTION_ID'][(df.ROOF_TYPE=='Tiles') & 
                                     (df.WALL_TYPE.isin([['Fibro / asbestos cement sheeting', 'Timber']])) &
                                     (df.YEAR_BUILT.isin(['1947 - 1961', '1962 - 1981']))] = 'dw309'

In [ ]:
# Start with simply assigning the modern building curve to all buildings, based on AS4055 classification.
# We'll overwrite specific groups at a later point. This is the quick option, since over 2/3rds of the 
# building population in the Cairns region is considered "modern" construction, so should conform 
# to AS4055 (i.e. constructed after 1981)

classes = ['C1', 'C2', 'C3', 'C4']
curves = ['dw317', 'dw318', 'dw319', 'dw320']
filter = df['YEAR_BUILT'].map(lambda x: x not in ['1982 - 1996', '1997 - present'])
for cls, curve in zip(classes, curves):
    idx = np.where(df['AS4055_CLASS'] == cls)[0]
    df['WIND_VULNERABILITY_FUNCTION_ID'][idx] = curve


In [ ]:
# Then work through the other options. Basically, its just a mapping of age, 
# roof type and wall type combinations to one of two options. 

# The data provided by JCU indicates a small proportion of buildings with construction era 1947-1952. The NEXIS YEAR_BUILT
# attribute has mapped this era to 1947-1961. This may generate some anomalies in the analysis. Re-mapping the source
# construction era to a new grouping in the NEXIS era would likely address this issue

# None in the Cairns NEXIS TCRM data (would have been mapped to Tile if any exist)
df['WIND_VULNERABILITY_FUNCTION_ID'][df['ROOF_TYPE']=='Concrete'] = 'dw316'
df['WIND_VULNERABILITY_FUNCTION_ID'][(df.ROOF_TYPE=='Fibro / asbestos cement sheeting') & 
                                     (df.YEAR_BUILT.isin(['1947 - 1961', '1962 - 1981']))] = 'dw315'
df['WIND_VULNERABILITY_FUNCTION_ID'][(df.ROOF_TYPE=='Fibro / asbestos cement sheeting') &
                                     (df.YEAR_BUILT.isin(['1840 - 1890', '1891 - 1913', '1914 - 1946']))] = 'dw316'
df['WIND_VULNERABILITY_FUNCTION_ID'][(df.ROOF_TYPE=='Metal Sheeting') & 
                                     (df.YEAR_BUILT.isin(['1840 - 1890', '1891 - 1913', '1914 - 1946','1947 - 1961', '1962 - 1981']))] = 'dw316'
df['WIND_VULNERABILITY_FUNCTION_ID'][(df.ROOF_TYPE=='Metal Sheeting') & 
                                     (df.YEAR_BUILT.isin(['1840 - 1890', '1891 - 1913', '1914 - 1946'])) & 
                                     (df.WALL_TYPE.isin(['Timber', 'Fibro / asbestos cement sheeting']))] = 'dw315'
df['WIND_VULNERABILITY_FUNCTION_ID'][(df.ROOF_TYPE=='Tiles') & 
                                     (df.YEAR_BUILT.isin(['1840 - 1890', '1891 - 1913', '1914 - 1946','1947 - 1961', '1962 - 1981']))] = 'dw315'
df['WIND_VULNERABILITY_FUNCTION_ID'][(df.ROOF_TYPE=='Tiles') & 
                                     (df.YEAR_BUILT.isin(['1840 - 1890', '1891 - 1913', '1914 - 1946'])) &
                                     (df.WALL_TYPE.isin(['Brick Veneer', 'Double Brick']))] = 'dw316'

In [ ]:
df.head()

In [ ]:
import seaborn as sns
sns.countplot(x='AS4055_CLASS', data=df,
              )

In [ ]:
df.to_csv("X:/georisk/HaRIA_B_Wind/projects/qfes_swha/data/derived/exposure/2020/SEQ_ResidentialExposure_NEXIS_2020_M4_updated.csv")

In [ ]:
df.AS4055_CLASS.unique()